In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr, norm
from tqdm import tqdm
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
chosen_countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 'Czechia', 'Estonia', 'Finland', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Korea', 'Latvia', 'Luxembourg', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Slovak Republic', 'Slovenia', 'Spain', 'Sweden', 'Türkiye']

life_expectancy_data = pd.read_csv('../data/life_expectancy.csv')
life_vars = life_expectancy_data["Variable"].unique().tolist()
pharma_sales_data = pd.read_csv('../data/pharma_sales_ppp.csv')
drug_vars = pharma_sales_data["Variable"].unique().tolist()

In [ ]:
def merge_and_plot(filtered_data_x, filtered_data_y, label_x, label_y):
    merged_data = pd.merge(filtered_data_x, filtered_data_y, on="Country", how="inner")
    selected_data = merged_data[["Country", "Value_x", "Value_y"]]
    selected_data.columns = ["Country", label_x, label_y]

    plt.scatter(selected_data[label_x], selected_data[label_y])
    plt.xlabel(label_x)
    plt.ylabel(label_y)
    plt.show()

def p_value_correlation(filtered_data_x, filtered_data_y, label_x, label_y, threshold):
    merged_data = pd.merge(filtered_data_x, filtered_data_y, on="Country", how="inner")
    selected_data = merged_data[["Country", "Value_x", "Value_y"]]

    if float(selected_data.corr(method="spearman")["Value_x"]["Value_y"]) >= threshold or selected_data.corr(method="spearman")["Value_x"]["Value_y"] <= -threshold:
        selected_data.columns = ["Country", label_x, label_y]


        # p value calculated in the spearman correlation is the correlation value
        # it is first transformed to a Z-score which normalizes it, so the data can
        # be seen as a normal distribution. The p-value is calculated with the cdf
        # and does a two tailed test. The test is done with a t-distribution
        corr, p_value = spearmanr(selected_data[label_x], selected_data[label_y])
        print("X:", label_x, "Y:", label_y)
        print("correlation between X and Y: ", corr)
        print("p value for correlation: ", p_value)


        if p_value <= 0.05:
            print("HYPOTHESIS REJECTED")

        else:
            print("HYPOTHESIS ACCEPTED")

def filter_data(data, year, variable, measure=None):
    if measure:
        return data[
            (data["Year"] == year) &
            (data["Country"].isin(chosen_countries)) &
            (data["Variable"] == variable) &
            (data["Measure"] == measure)
        ]
    else:
        return data[
            (data["Year"] == year) &
            (data["Country"].isin(chosen_countries)) &
            (data["Variable"] == variable)
        ]



In [ ]:


counter = 0
for drug in drug_vars:

    filtered_pharma_sales = filter_data(pharma_sales_data, 2014, drug)

    for life in life_vars:
        filtered_life_expectancy = filter_data(life_expectancy_data, 2014, life, "Years")

        # merge_and_plot(filtered_pharma_sales, filtered_life_expectancy, str(drug), str(life))
        p_value_correlation(filtered_pharma_sales, filtered_life_expectancy, str(drug), str(life), 0.4)


In [ ]:
drug_vars1 = drug_vars
drug_vars2 = drug_vars

for drug1 in drug_vars1:
    filtered_pharma_sales1 = filter_data(pharma_sales_data, 2014, drug1)

    for drug2 in drug_vars2:
        filtered_pharma_sales2 = filter_data(pharma_sales_data, 2014, drug2)
        if drug1 != drug2:

            # merge_and_plot(filtered_pharma_sales1, filtered_pharma_sales2, str(drug_vars1), str(drug_vars2))
            p_value_correlation(filtered_pharma_sales1, filtered_pharma_sales2, str(drug_vars1), str(drug_vars2), 0.4)

In [ ]:
alcohol_consumption_data = pd.read_csv('../data/alcohol_consump.csv')
alcohol_measures = alcohol_consumption_data["Measure"].unique().tolist()

for alcohol in alcohol_measures:
    filtered_alcohol_consumption = filter_data(alcohol_consumption_data, 2014, "Alcohol consumption", alcohol)

    for life in life_vars:
        filtered_life_expectancy = filter_data(life_expectancy_data, 2014, life, "Years")

        # merge_and_plot(filtered_alcohol_consumption, filtered_life_expectancy, str(alcohol), str(life))
        p_value_correlation(filtered_alcohol_consumption, filtered_life_expectancy, str(alcohol), str(life), 0.4)

In [ ]:
# food data werkt nog niet

food_data = pd.read_csv('../data/food.csv')
food_measures = food_data["Measure"].unique().tolist()
food_variables = food_data["Variable"].unique().tolist()

for food in food_variables:
    filtered_food_data = filter_data(food_data, 2014, )

    food_data[
        (food_data["Year"] == 2014) &
        (food_data["Country"].isin(chosen_countries)) &
        (food_data["Measure"] == food)
    ]
    for life in life_vars:
        filtered_life_expectancy = life_expectancy_data[
        (life_expectancy_data["Year"] == 2014) &
        (life_expectancy_data["Measure"] == "Years") &
        (life_expectancy_data["Country"].isin(chosen_countries)) &
        (life_expectancy_data["Variable"] == life)
        ]

        scatter_plot(filtered_food_data, filtered_life_expectancy, str(food), str(life), 0.0)

In [ ]:
# dit moet ook herschreven worden

from sklearn import linear_model
from sklearn.linear_model import LinearRegression

variables = ["A02B-Drugs for peptic ulcer and gastro-oesophageal reflux diseases (GORD)", "N-Nervous system", "N06A-Antidepressants"]
estimate = ["Females at age 40", "Males at age 40"]
filtered_pharma_sales = pharma_sales_data[
    (pharma_sales_data["Year"] == 2014) &
    (pharma_sales_data["Country"].isin(chosen_countries)) &
    (pharma_sales_data["Variable"].isin(variables))
]
# print(filtered_pharma_sales)
df = defaultdict(lambda: [])
for elem in filtered_pharma_sales[["Variable", "Value"]].iterrows():
    df[elem[1]["Variable"]].append(elem[1]["Value"])

df = pd.DataFrame(df)

X = df[["A02B-Drugs for peptic ulcer and gastro-oesophageal reflux diseases (GORD)", "N-Nervous system", "N06A-Antidepressants"]]

filtered_life_expectancy = filter_data(life_expectancy_data, 2014, estimate[0], "Years")

y = filtered_life_expectancy["Value"]

model = LinearRegression()
model.fit(X, y)
print(model.intercept_)
print(model.coef_)





